# 3_compute_embeddings

Genera embeddings de texto e imagen a partir de `products_clean`.
Este notebook es tolerante a dependencias ausentes (cae a embeddins simulados y salta parquet si no hay pyarrow).

In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
try:
    from tqdm.auto import tqdm
except ImportError:
    print("⚠️ tqdm no instalado; sin barra de progreso. `pip install tqdm` para activarla.")
    tqdm = lambda x, **k: x

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
STEP2_DIR = PROJECT_ROOT / "notebooks" / "data" / "step_2"
FINAL_PARQUET = PROJECT_ROOT / "data" / "products_with_embeddings.parquet"
FINAL_JSONL = PROJECT_ROOT / "data" / "products_with_embeddings.jsonl"
IMAGES_CACHE = PROJECT_ROOT / "notebooks" / "data" / "images_cache"
IMAGES_CACHE.mkdir(parents=True, exist_ok=True)

candidates = sorted(STEP2_DIR.glob("products_clean_*.parquet")) or sorted(STEP2_DIR.glob("products_clean_*.jsonl"))
if not candidates:
    raise FileNotFoundError(f"No hay products_clean en {STEP2_DIR}. Ejecuta 2_preparacion_productos.ipynb.")
INPUT_PATH = candidates[0]
print(f"Usando {INPUT_PATH.name}")

if INPUT_PATH.suffix == ".parquet":
    df = pd.read_parquet(INPUT_PATH)
else:
    records = [json.loads(line) for line in INPUT_PATH.read_text(encoding="utf-8").splitlines() if line.strip()]
    df = pd.DataFrame.from_records(records)
df.head()

Usando products_clean_Cell_Phones_and_Accessories_sample.jsonl


/Users/marc/Documents/Projectes/similaritify/.env_machine_learning/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,title,description,category_path,image_url,text_for_embedding
0,0,Pink &amp; White 3d Melt Ice-cream Skin Hard C...,Pink & White 3D Melt Ice-Cream Skin Hard Case ...,,https://ecx.images-amazon.com/images/I/31zn6SO...,Pink &amp; White 3d Melt Ice-cream Skin Hard C...
1,1,Purple Hard Case Cover for Iphone 4 4s 4g with...,Purple Hard Case Cover for iPhone 4 4S 4G With...,,https://ecx.images-amazon.com/images/I/41WCZc2...,Purple Hard Case Cover for Iphone 4 4s 4g with...
2,2,Hello Kitty Light-weighted Chrome Case Black C...,"Thin and light weighted, Case's unique design ...",,https://ecx.images-amazon.com/images/I/41fy1%2...,Hello Kitty Light-weighted Chrome Case Black C...
3,3,Cool Summer Breeze in the Ocean Beach Collecti...,Product Name: Cool Summer Breeze In The Ocean ...,,https://ecx.images-amazon.com/images/I/415cmp6...,Cool Summer Breeze in the Ocean Beach Collecti...
4,4,Cool Summer Breeze In The Ocean Beach Collecti...,Product Name: Cool Summer Breeze In The Ocean ...,,https://ecx.images-amazon.com/images/I/41XDwPt...,Cool Summer Breeze In The Ocean Beach Collecti...


## Embeddings de texto (fallback si no hay sentence-transformers)

In [2]:
try:
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    texts = df["text_for_embedding"].fillna("").tolist()
    emb_text = model.encode(texts, batch_size=64, convert_to_numpy=True, show_progress_bar=True, normalize_embeddings=True)
    print("emb_text shape:", emb_text.shape)
except Exception as e:
    print("⚠️ sentence-transformers no disponible, usando emb_text simulados (ceros)")
    print(e)
    emb_text = np.zeros((len(df), 1), dtype="float32")

Batches: 100%|██████████| 32/32 [00:04<00:00,  6.86it/s]

emb_text shape: (2000, 384)


## Embeddings de imagen (opcional; fallback a ceros)

In [3]:
emb_img = np.zeros((len(df), 1), dtype="float32")  # fallback por defecto
try:
    import io, requests
    from PIL import Image
    from sentence_transformers import SentenceTransformer
    img_model = SentenceTransformer("clip-ViT-L-14")

    def load_image(url: str):
        try:
            resp = requests.get(url, timeout=10)
            resp.raise_for_status()
            return Image.open(io.BytesIO(resp.content)).convert("RGB")
        except Exception:
            return Image.new("RGB", (224, 224), color=(0, 0, 0))

    images = [load_image(url) for url in tqdm(df["image_url"].fillna("").tolist(), desc="img")]
    emb_img = img_model.encode(images, batch_size=32, convert_to_numpy=True, show_progress_bar=True, normalize_embeddings=True)
    print("emb_img shape:", emb_img.shape)
except Exception as e:
    print("⚠️ Embeddings de imagen omitidos (usa fallback de ceros). Instala pillow/requests/sentence-transformers si quieres usarlos.")
    print(e)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Batches: 100%|██████████| 63/63 [02:35<00:00,  2.47s/it]

emb_img shape: (2000, 768)


## Guardado (JSONL siempre; parquet si pyarrow está disponible)

In [4]:
# Asegurar tipos serializables
df_out = df.copy()
df_out["emb_text"] = list(emb_text)
df_out["emb_img"] = list(emb_img)

parquet_ok = False
try:
    import pyarrow  # noqa: F401
    df_out.to_parquet(FINAL_PARQUET, index=False)
    parquet_ok = True
except Exception as e:
    print("⚠️ Parquet no guardado; se continua con JSONL/NPY.")
    print(e)

# Serializar embeddings a listas para JSON
df_json = df_out.copy()
df_json["emb_text"] = df_json["emb_text"].apply(lambda x: x.tolist() if hasattr(x, "tolist") else list(x))
df_json["emb_img"] = df_json["emb_img"].apply(lambda x: x.tolist() if hasattr(x, "tolist") else list(x))

with open(FINAL_JSONL, "w", encoding="utf-8") as f:
    for rec in df_json.to_dict(orient="records"):
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

# Guardado npy
np.save(PROJECT_ROOT / "data" / "emb_text.npy", np.array(df_json["emb_text"].tolist()))
np.save(PROJECT_ROOT / "data" / "emb_img.npy", np.array(df_json["emb_img"].tolist()))

if parquet_ok:
    print("Guardado Parquet:", FINAL_PARQUET)
else:
    print("Parquet omitido")
print("Guardado JSONL:", FINAL_JSONL)


Guardado Parquet: /Users/marc/Documents/Projectes/similaritify/data/products_with_embeddings.parquet
Guardado JSONL: /Users/marc/Documents/Projectes/similaritify/data/products_with_embeddings.jsonl
